We will start processing the news data in this notebook. I have done some work here -
1. Added symbol column (my mistake while building the dataset and then my api tokens got empty)
2. Translated all the useful data into english for analysis (done through google sheets - might implement it here through API or would fix the language to english in news api)

In [51]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
news_data = pd.read_csv('/home/puskchan/Stock_project/data/semi_cleaned_articles.csv')

In [53]:
news_data.shape

(5414, 8)

In [54]:
news_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5414 entries, 0 to 5413
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   clean_url       5414 non-null   object
 1   country         5414 non-null   object
 2   excerpt         5414 non-null   object
 3   published_date  5414 non-null   object
 4   title           5414 non-null   object
 5   symbol          5407 non-null   object
 6   title_en        5414 non-null   object
 7   excerpt_en      5414 non-null   object
dtypes: object(8)
memory usage: 338.5+ KB


In [55]:
news_data.isna().sum()

clean_url         0
country           0
excerpt           0
published_date    0
title             0
symbol            7
title_en          0
excerpt_en        0
dtype: int64

In [56]:
news_data.drop_duplicates(subset=['title_en', 'excerpt_en'], inplace=True)
news_data.dropna(subset=['excerpt_en', 'title_en', 'symbol'], inplace=True)  # Ensure no missing text


In [57]:
news_data.shape

(4339, 8)

In [58]:
news_data['excerpt_en'] = news_data['excerpt_en'].str.lower()
news_data['title_en'] = news_data['title_en'].str.lower()


In [59]:
import re
news_data['excerpt_en'] = news_data['excerpt_en'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))
news_data['title_en'] = news_data['title_en'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))

In [60]:
news_data['word_string'] = news_data['excerpt_en'] + news_data['title_en']

In [61]:
nltk.download('punkt')
nltk.download('punkt_tab')


news_data['tokens'] = news_data['word_string'].apply(word_tokenize)


[nltk_data] Downloading package punkt to /home/puskchan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/puskchan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [62]:
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
news_data['tokens'] = news_data['tokens'].apply(lambda x: [word for word in x if word not in stop_words])


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/puskchan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
news_data['tokens'] = news_data['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


[nltk_data] Downloading package wordnet to /home/puskchan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [64]:
news_data['cleaned_excerpt'] = news_data['tokens'].apply(lambda x: ' '.join(x))

In [65]:
news_data.reset_index(drop=True, inplace=True)

In [67]:
news_data.head()

,clean_url,country,excerpt,published_date,title,symbol,title_en,excerpt_en,word_string,tokens,cleaned_excerpt
0,zonebourse.com,FR,Nomura est positif et recommande le titre à l'...,2024-04-29 10:47:12,Bajaj Finserv Ltd. : Nomura persiste à l'achat,BAJFINANCE.NS,bajaj finserv ltd nomura persists with the pu...,nomura is positive and recommends the stock fo...,nomura is positive and recommends the stock fo...,"[nomura, positive, recommends, stock, purchase...",nomura positive recommends stock purchase prev...
1,livemint.com,IN,"Stocks at record high: Bharti Airtel, HCL Tech...",2024-09-13 10:20:09,"Stocks at record high: Bharti Airtel, HCL Tech...",BHARTIARTL.NS,stocks at record high bharti airtel hcl tech b...,stocks at record high bharti airtel hcl tech b...,stocks at record high bharti airtel hcl tech b...,"[stock, record, high, bharti, airtel, hcl, tec...",stock record high bharti airtel hcl tech bajaj...
2,moneycontrol.com,IN,"The sector is consolidating, with larger playe...",2024-09-25 14:26:00,Daily Voice: Real estate in middle of upcycle ...,BAJFINANCE.NS,daily voice real estate in middle of upcycle a...,the sector is consolidating with larger player...,the sector is consolidating with larger player...,"[sector, consolidating, larger, player, gainin...",sector consolidating larger player gaining mar...
3,coatingsworld.com,US,New CPVC resin plant in Vilayat advances Lubri...,2023-11-22 10:00:44,Lubrizol and Grasim Industries Break Ground on...,GRASIM.NS,lubrizol and grasim industries break ground on...,new cpvc resin plant in vilayat advances lubri...,new cpvc resin plant in vilayat advances lubri...,"[new, cpvc, resin, plant, vilayat, advance, lu...",new cpvc resin plant vilayat advance lubrizols...
4,thehansindia.com,IN,Indian equity indices closed flat on Tuesday a...,2024-09-03 11:11:40,"Sensex closes flat, ICICI Bank and Bajaj Finse...",ICICIBANK.NS,sensex closes flat icici bank and bajaj finser...,indian equity indices closed flat on tuesday a...,indian equity indices closed flat on tuesday a...,"[indian, equity, index, closed, flat, tuesday,...",indian equity index closed flat tuesday amid m...


In [68]:
news_data.to_csv("cleaned_articles.csv", index=False)